In [ ]:
import json
from ipyleaflet import Map, GeoJSON

from rgd_watch_client import create_watch_client

# Login

Login into the RGD WATCH production instance using the `rgd_watch_client` package and `create_watch_client`. We are working to make this `pip` installable from the [RD-WACTH](https://github.com/ResonantGeoData/RD-WATCH) repository's GitHub Pages site. At present, a wheel is in the build artifacts for [this CI pipeline](https://github.com/ResonantGeoData/RD-WATCH/actions/workflows/ci.yml).

In [ ]:
client = create_watch_client(username='bane.sullivan@kitware.com', )
client

# Perfrom a Search

We can use a GeoJSON polygon to perform a spatial search:

In [ ]:
bbox = {"type":"Polygon","coordinates":[[[-45.45413076707502,-24.337204409169036],
                                         [-45.45413076707502,-20.600008202090393],
                                         [-40.601886755483115,-20.600008202090393],
                                         [-40.601886755483115,-24.337204409169036],
                                         [-45.45413076707502,-24.337204409169036]]]}

q = client.rgd.search(query=json.dumps(bbox), predicate='within', collections=['WorldView'])
len(q['results'])

In [ ]:
m = Map(zoom=7, center=[-22.504764632039553, -42.73127563297749])

# Add search geometry to map
m.add_layer(GeoJSON(data=bbox, style={'fillOpacity': 0}))

for s in q['results']:
    m.add_layer(GeoJSON(data=s['outline']))

m

# Inspect a Raster from Search Results

In [ ]:
result = q['results'][0]
result

In [ ]:
raster = client.imagery.get_raster(result)
raster

## Get STAC Item format

In [ ]:
stac = client.imagery.get_raster(result, stac=True)
stac

## Stream Tiles for Local Visualization

In [ ]:
im = raster['parent_raster']['image_set']['images'][0]
im

In [ ]:
m = Map(zoom=11, center=[-22.654202783231515, -43.21436335332692])

tiles = client.imagery.get_leaflet_tile_source(im['id'])
m.add_layer(tiles)
m

## Extract Region of Interest

In [ ]:
roi = {"type":"Polygon","coordinates":[[[-43.2808197484453,-22.731514330238422],[-43.2808197484453,-22.694639112329988],[-43.23286860251871,-22.694639112329988],[-43.23286860251871,-22.731514330238422],[-43.2808197484453,-22.731514330238422]]]}

In [ ]:
m = Map(zoom=11, center=[-22.654202783231515, -43.21436335332692])

tiles = client.imagery.get_leaflet_tile_source(im['id'])
m.add_layer(tiles)
m.add_layer(GeoJSON(data=roi))
m

In [ ]:
# Create a Processing group
params = {'sample_type': 'geojson', 'projection': 'EPSG:3857'}
params.update(roi)
group = client.imagery.create_processed_image_group('region', params)

In [ ]:
# Process the single image
processed = client.imagery.create_processed_image([im['id']], group)

In [ ]:
# Check status
client.imagery.get_processed_image_group_status(group)

# Post new data

In [ ]:
# post_stac_file